In [ ]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

: 

In [ ]:

class BrapiApi:
    def __init__(self, base_url=os.getenv("VITE_BRAPI_BASE_URL")):
        self.base_url = base_url

    def get_quote_by_name(self, name):
        url = f"{self.base_url}/quote/{name}"   
        response = requests.get(url, headers={
           'Authorization': f'Bearer {os.getenv("VITE_BRAPI_AUTH_TOKEN")}'
        })
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch price for {name}. Status code: {response.status_code}")
            return None


class FinancingApi:
    def __init__(self, base_url=os.getenv("VITE_FINANCING_BASE_URL")):
        self.base_url = base_url

    def post_price(self, investiment_id, price):
        url = f"{self.base_url}/prices"
        payload = {
            "investiment_id": investiment_id,
            "price": price
        }
        response = requests.post(url, json=payload)
        if response.status_code == 201:
            print(f"Price posted successfully for {investiment_id}")
        else:
            print(f"Failed to post price for {investiment_id}. Status code: {response.status_code}")

    def put_price_by_id(self, investiment_id, price):
        url = f"{self.base_url}/prices/{investiment_id}"
        payload = {
            "price": price
        }
        response = requests.put(url, json=payload)
        if response.status_code == 200:
            print(f"Price updated successfully for {investiment_id}")
        else:
            print(f"Failed to update price for {investiment_id}. Status code: {response.status_code}")


def update_prices_with_brapi(investiments, investiment_type):
    brapi_api_service = BrapiApi()
    financing_api_service = FinancingApi()

    for inv in investiments:
        if inv["investiment_type"] != investiment_type: continue
        brapi_price = brapi_api_service.get_quote_by_name(inv["name"])
        if brapi_price and brapi_price.get("results") and brapi_price["results"][0].get("regularMarketPrice"):
            price = brapi_price["results"][0]["regularMarketPrice"]
            print(price)
            financing_api_service.post_price(inv["id"], price)
        else:
            print(f"Failed to fetch price for {inv['name']}")


investiments = requests.get(f"{os.getenv('VITE_FINANCING_BASE_URL')}/investiments").json()
investiment_type = "FII"
update_prices_with_brapi(investiments, investiment_type)